In [1]:
%load_ext autoreload
%autoreload 2

# Semantic Knowledge

This will process the transcripts/annotations (which would be considered autobiographical knowledge) to extract the meaning of the emotion words, i.e. semantic knowledge, as subclass of an nltk naive Bayes classifier
- the features/observations/data are a set of (question, answer) pairs
- the models/hypotheses are the emotion words
- the data gives us P(observation|emotion word)
- the task goal is to calculate P(emotion word|observation)
- the prior P(emotion word) = 1/(# of emotion words), i.e. a uniform, non-informative prior

In [24]:
from emo20q.semanticknowledge import SemanticKnowledge
k = SemanticKnowledge()

In [25]:
print(k.entities())

{'uninterested', 'curiosity', 'caring', 'scared', 'unsure', 'annoyance', 'let down', 'disappointment', 'sorrow', 'dumbfounded', 'aversion', 'insecurity', 'hope', 'admiration', 'jealousy', 'delirium', 'respect', 'conflicted', 'jubilance', 'adoration', 'surprise', 'stress', 'apathy', 'avarice', 'determination', 'relief', 'fear', 'happiness', 'hunger', 'euphoria', 'thrill', 'nostalgia', 'coldness', 'bravery', 'serenity', 'thankfulness', 'pity', 'pessimism', 'remorse', 'glee', 'annoyed', 'infuriation', 'exotic', 'downcast', 'confidence', 'ecstasy', 'obsession', 'tense', 'sleepiness', 'depression', 'frustration', 'relaxation', 'maudlin', 'upset', 'love', 'disgust', 'loneliness', 'exhaustion', 'ambivalence', 'aggravation', 'shyness', 'jilted', 'hopelessness', 'disillusionment', 'joyfulness', 'wariness', 'guilt', 'indifference', 'boredom', 'ambiguity', 'delight', 'accepting', 'maniacal', 'loved', 'gratefulness', 'being appalled', 'educated', 'cheerfulness', 'courage', 'discontentment', 'worry

In [26]:
print(k.most_informative_features(20))

[('e.valence==positive', 'no'), ('e==happiness', 'no'), ('e.valence==negative', 'yes'), ('associated(e,happiness)', 'no'), ('e.activation==high', 'no'), ('e.activation==high', 'other'), ('cause(e,feelGood)', 'other'), ('e.valence==negative', 'no'), ('e.duration==long', 'no'), ('e.valence==positive', 'other'), ('e==happiness', 'yes'), ('e==depression', 'yes'), ('cause(something,e)', 'no'), ('directed(e,otherPerson)', 'yes'), ('directed(e,otherPerson)', 'other'), ('directed(e,otherPerson)', 'no'), ('e.duration==long', 'other'), ('cause(something,e)', 'other'), ('similar(e,happiness)', 'other'), ('e==hope', 'yes')]
